# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [6]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [7]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_sentence = [sentence for sentence in source_text.split('\n')]    
    target_sentence = [sentence + ' <EOS> ' for sentence in target_text.split('\n')]    
    source_id = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_sentence]   
    target_id = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in target_sentence]  
    return source_id, target_id

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [11]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, (None, None), name='input')
    targets = tf.placeholder(tf.int32, (None, None), name='targets')
    learning_rate = tf.placeholder(tf.float32, None)
    keep_probability = tf.placeholder(tf.float32, None, name='keep_prob')
    source_length = tf.placeholder(tf.int32, [None], name='source_sequence_length')
    target_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_sequence = tf.reduce_max(target_length, name='max_target_len')
    return input, targets, learning_rate, keep_probability, target_length, max_target_sequence, source_length 

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/local/lib/python3.5/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/local/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'F

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [12]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    go = target_vocab_to_int['<GO>']
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    return tf.concat([tf.fill([batch_size, 1], go), ending], 1)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [13]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    inputs = tf.contrib.layers.embed_sequence(rnn_inputs,source_vocab_size,encoding_embedding_size)
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(rnn_size), output_keep_prob=keep_prob) for _ in range(num_layers)])
    output, rnn_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    return output, rnn_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [14]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    train_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    dec_fn_train = tf.contrib.seq2seq.BasicDecoder(dec_cell, train_helper, encoder_state, output_layer)
    training_logits, state, _ = tf.contrib.seq2seq.dynamic_decode(dec_fn_train, impute_finished=True, maximum_iterations=max_summary_length)
    return training_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [15]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    with tf.variable_scope("decode", reuse=None):
        start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], 
                               name='start_tokens')
        # Helper for the inference process.
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, end_of_sequence_id)
        # Basic decoder
        inference_decoder  = tf.contrib.seq2seq.BasicDecoder(dec_cell, inference_helper, encoder_state, output_layer)                                               
        # Perform dynamic decoding using the decoder
        inference_decoder_output, _, _  = tf.contrib.seq2seq.dynamic_decode(inference_decoder, impute_finished=True, maximum_iterations=max_target_sequence_length)
    return inference_decoder_output
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [16]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder Embedding
        
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    # 2. Construct the decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size, kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    # 4. Set up a training decoder and an inference decoder
    with tf.variable_scope("decode") as decoding_scope:
        # Training Decoder
        training_decoder_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, 
                         output_layer, keep_prob)
        decoding_scope.reuse_variables()
        # 5. Inference Decoder
        # Reuses the same parameters trained by the training process
        inference_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'],
                             target_vocab_to_int['<EOS>'], max_target_sequence_length,
                             target_vocab_size, output_layer, batch_size, keep_prob)
    return training_decoder_output, inference_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [17]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    enc_output, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    training_logits, inference_logits = decoding_layer(dec_input,
                                                   enc_state,
                                                   target_sequence_length,
                                                   max_target_sentence_length,
                                                   rnn_size, 
                                                   num_layers,
                                                   target_vocab_to_int,
                                                   target_vocab_size,
                                                   batch_size,
                                                   keep_prob,
                                                   dec_embedding_size)
    return training_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [18]:
# Number of Epochs
epochs = 7
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 196
decoding_embedding_size = 196
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.7
display_step = 2

### Build the Graph
Build the graph using the neural network you implemented.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    2/538 - Train Accuracy: 0.2516, Validation Accuracy: 0.3157, Loss: 4.6321
Epoch   0 Batch    4/538 - Train Accuracy: 0.2699, Validation Accuracy: 0.3450, Loss: 4.3603
Epoch   0 Batch    6/538 - Train Accuracy: 0.3047, Validation Accuracy: 0.3469, Loss: 3.8930
Epoch   0 Batch    8/538 - Train Accuracy: 0.2809, Validation Accuracy: 0.3475, Loss: 3.6552
Epoch   0 Batch   10/538 - Train Accuracy: 0.2723, Validation Accuracy: 0.3571, Loss: 3.5004
Epoch   0 Batch   12/538 - Train Accuracy: 0.3285, Validation Accuracy: 0.4020, Loss: 3.3210
Epoch   0 Batch   14/538 - Train Accuracy: 0.3318, Validation Accuracy: 0.3979, Loss: 3.1017
Epoch   0 Batch   16/538 - Train Accuracy: 0.3620, Validation Accuracy: 0.4057, Loss: 2.8876
Epoch   0 Batch   18/538 - Train Accuracy: 0.3506, Validation Accuracy: 0.4189, Loss: 2.9652
Epoch   0 Batch   20/538 - Train Accuracy: 0.3904, Validation Accuracy: 0.4308, Loss: 2.7279
Epoch   0 Batch   22/538 - Train Accuracy: 0.3850, Validation Accuracy

Epoch   0 Batch  180/538 - Train Accuracy: 0.5091, Validation Accuracy: 0.5277, Loss: 1.0644
Epoch   0 Batch  182/538 - Train Accuracy: 0.4684, Validation Accuracy: 0.5316, Loss: 1.0972
Epoch   0 Batch  184/538 - Train Accuracy: 0.5035, Validation Accuracy: 0.5391, Loss: 1.0144
Epoch   0 Batch  186/538 - Train Accuracy: 0.4883, Validation Accuracy: 0.5297, Loss: 1.0356
Epoch   0 Batch  188/538 - Train Accuracy: 0.4801, Validation Accuracy: 0.5353, Loss: 1.0450
Epoch   0 Batch  190/538 - Train Accuracy: 0.5164, Validation Accuracy: 0.5568, Loss: 1.0508
Epoch   0 Batch  192/538 - Train Accuracy: 0.4987, Validation Accuracy: 0.5293, Loss: 1.0021
Epoch   0 Batch  194/538 - Train Accuracy: 0.4857, Validation Accuracy: 0.5403, Loss: 1.0398
Epoch   0 Batch  196/538 - Train Accuracy: 0.5206, Validation Accuracy: 0.5508, Loss: 0.9923
Epoch   0 Batch  198/538 - Train Accuracy: 0.5326, Validation Accuracy: 0.5369, Loss: 0.9625
Epoch   0 Batch  200/538 - Train Accuracy: 0.5209, Validation Accuracy

Epoch   0 Batch  358/538 - Train Accuracy: 0.6014, Validation Accuracy: 0.6289, Loss: 0.6304
Epoch   0 Batch  360/538 - Train Accuracy: 0.6238, Validation Accuracy: 0.6401, Loss: 0.6407
Epoch   0 Batch  362/538 - Train Accuracy: 0.6257, Validation Accuracy: 0.6362, Loss: 0.5969
Epoch   0 Batch  364/538 - Train Accuracy: 0.5828, Validation Accuracy: 0.6207, Loss: 0.6595
Epoch   0 Batch  366/538 - Train Accuracy: 0.6295, Validation Accuracy: 0.6321, Loss: 0.6298
Epoch   0 Batch  368/538 - Train Accuracy: 0.6758, Validation Accuracy: 0.6380, Loss: 0.5536
Epoch   0 Batch  370/538 - Train Accuracy: 0.6021, Validation Accuracy: 0.6394, Loss: 0.6362
Epoch   0 Batch  372/538 - Train Accuracy: 0.6318, Validation Accuracy: 0.6387, Loss: 0.6213
Epoch   0 Batch  374/538 - Train Accuracy: 0.6004, Validation Accuracy: 0.6333, Loss: 0.6308
Epoch   0 Batch  376/538 - Train Accuracy: 0.6090, Validation Accuracy: 0.6335, Loss: 0.6317
Epoch   0 Batch  378/538 - Train Accuracy: 0.6257, Validation Accuracy

Epoch   0 Batch  536/538 - Train Accuracy: 0.6808, Validation Accuracy: 0.6786, Loss: 0.4744
Epoch   1 Batch    2/538 - Train Accuracy: 0.6508, Validation Accuracy: 0.6776, Loss: 0.4938
Epoch   1 Batch    4/538 - Train Accuracy: 0.6660, Validation Accuracy: 0.6889, Loss: 0.4707
Epoch   1 Batch    6/538 - Train Accuracy: 0.6784, Validation Accuracy: 0.6951, Loss: 0.4523
Epoch   1 Batch    8/538 - Train Accuracy: 0.6727, Validation Accuracy: 0.6836, Loss: 0.4647
Epoch   1 Batch   10/538 - Train Accuracy: 0.6525, Validation Accuracy: 0.6944, Loss: 0.4812
Epoch   1 Batch   12/538 - Train Accuracy: 0.6471, Validation Accuracy: 0.6836, Loss: 0.4738
Epoch   1 Batch   14/538 - Train Accuracy: 0.6779, Validation Accuracy: 0.6914, Loss: 0.4618
Epoch   1 Batch   16/538 - Train Accuracy: 0.6801, Validation Accuracy: 0.6951, Loss: 0.4386
Epoch   1 Batch   18/538 - Train Accuracy: 0.6381, Validation Accuracy: 0.6928, Loss: 0.4802
Epoch   1 Batch   20/538 - Train Accuracy: 0.6749, Validation Accuracy

Epoch   1 Batch  178/538 - Train Accuracy: 0.7710, Validation Accuracy: 0.7743, Loss: 0.2826
Epoch   1 Batch  180/538 - Train Accuracy: 0.8008, Validation Accuracy: 0.7814, Loss: 0.2875
Epoch   1 Batch  182/538 - Train Accuracy: 0.7727, Validation Accuracy: 0.7555, Loss: 0.2927
Epoch   1 Batch  184/538 - Train Accuracy: 0.7628, Validation Accuracy: 0.7603, Loss: 0.2751
Epoch   1 Batch  186/538 - Train Accuracy: 0.7807, Validation Accuracy: 0.7594, Loss: 0.2812
Epoch   1 Batch  188/538 - Train Accuracy: 0.7936, Validation Accuracy: 0.7757, Loss: 0.2837
Epoch   1 Batch  190/538 - Train Accuracy: 0.7658, Validation Accuracy: 0.7727, Loss: 0.3061
Epoch   1 Batch  192/538 - Train Accuracy: 0.7917, Validation Accuracy: 0.7800, Loss: 0.2712
Epoch   1 Batch  194/538 - Train Accuracy: 0.7617, Validation Accuracy: 0.7729, Loss: 0.2835
Epoch   1 Batch  196/538 - Train Accuracy: 0.7857, Validation Accuracy: 0.7795, Loss: 0.2706
Epoch   1 Batch  198/538 - Train Accuracy: 0.8119, Validation Accuracy

...

Epoch   5 Batch  348/538 - Train Accuracy: 0.9701, Validation Accuracy: 0.9540, Loss: 0.0157
Epoch   5 Batch  350/538 - Train Accuracy: 0.9762, Validation Accuracy: 0.9572, Loss: 0.0206
Epoch   5 Batch  352/538 - Train Accuracy: 0.9606, Validation Accuracy: 0.9632, Loss: 0.0297
Epoch   5 Batch  354/538 - Train Accuracy: 0.9770, Validation Accuracy: 0.9695, Loss: 0.0155
Epoch   5 Batch  356/538 - Train Accuracy: 0.9778, Validation Accuracy: 0.9641, Loss: 0.0156
Epoch   5 Batch  358/538 - Train Accuracy: 0.9793, Validation Accuracy: 0.9560, Loss: 0.0147
Epoch   5 Batch  360/538 - Train Accuracy: 0.9787, Validation Accuracy: 0.9673, Loss: 0.0146
Epoch   5 Batch  362/538 - Train Accuracy: 0.9794, Validation Accuracy: 0.9645, Loss: 0.0133
Epoch   5 Batch  364/538 - Train Accuracy: 0.9652, Validation Accuracy: 0.9638, Loss: 0.0214
Epoch   5 Batch  366/538 - Train Accuracy: 0.9697, Validation Accuracy: 0.9565, Loss: 0.0167
Epoch   5 Batch  368/538 - Train Accuracy: 0.9806, Validation Accuracy

Epoch   5 Batch  526/538 - Train Accuracy: 0.9762, Validation Accuracy: 0.9583, Loss: 0.0172
Epoch   5 Batch  528/538 - Train Accuracy: 0.9875, Validation Accuracy: 0.9631, Loss: 0.0165
Epoch   5 Batch  530/538 - Train Accuracy: 0.9598, Validation Accuracy: 0.9581, Loss: 0.0202
Epoch   5 Batch  532/538 - Train Accuracy: 0.9645, Validation Accuracy: 0.9540, Loss: 0.0144
Epoch   5 Batch  534/538 - Train Accuracy: 0.9855, Validation Accuracy: 0.9656, Loss: 0.0116
Epoch   5 Batch  536/538 - Train Accuracy: 0.9823, Validation Accuracy: 0.9700, Loss: 0.0201
Epoch   6 Batch    2/538 - Train Accuracy: 0.9818, Validation Accuracy: 0.9698, Loss: 0.0168
Epoch   6 Batch    4/538 - Train Accuracy: 0.9725, Validation Accuracy: 0.9730, Loss: 0.0169
Epoch   6 Batch    6/538 - Train Accuracy: 0.9741, Validation Accuracy: 0.9739, Loss: 0.0154
Epoch   6 Batch    8/538 - Train Accuracy: 0.9754, Validation Accuracy: 0.9748, Loss: 0.0154
Epoch   6 Batch   10/538 - Train Accuracy: 0.9832, Validation Accuracy

Epoch   6 Batch  168/538 - Train Accuracy: 0.9680, Validation Accuracy: 0.9656, Loss: 0.0185
Epoch   6 Batch  170/538 - Train Accuracy: 0.9693, Validation Accuracy: 0.9570, Loss: 0.0166
Epoch   6 Batch  172/538 - Train Accuracy: 0.9821, Validation Accuracy: 0.9682, Loss: 0.0125
Epoch   6 Batch  174/538 - Train Accuracy: 0.9834, Validation Accuracy: 0.9609, Loss: 0.0129
Epoch   6 Batch  176/538 - Train Accuracy: 0.9750, Validation Accuracy: 0.9698, Loss: 0.0174
Epoch   6 Batch  178/538 - Train Accuracy: 0.9604, Validation Accuracy: 0.9688, Loss: 0.0175
Epoch   6 Batch  180/538 - Train Accuracy: 0.9727, Validation Accuracy: 0.9647, Loss: 0.0144
Epoch   6 Batch  182/538 - Train Accuracy: 0.9805, Validation Accuracy: 0.9656, Loss: 0.0131
Epoch   6 Batch  184/538 - Train Accuracy: 0.9782, Validation Accuracy: 0.9670, Loss: 0.0142
Epoch   6 Batch  186/538 - Train Accuracy: 0.9734, Validation Accuracy: 0.9666, Loss: 0.0146
Epoch   6 Batch  188/538 - Train Accuracy: 0.9662, Validation Accuracy

Epoch   6 Batch  346/538 - Train Accuracy: 0.9753, Validation Accuracy: 0.9631, Loss: 0.0156
Epoch   6 Batch  348/538 - Train Accuracy: 0.9704, Validation Accuracy: 0.9629, Loss: 0.0131
Epoch   6 Batch  350/538 - Train Accuracy: 0.9805, Validation Accuracy: 0.9599, Loss: 0.0168
Epoch   6 Batch  352/538 - Train Accuracy: 0.9701, Validation Accuracy: 0.9631, Loss: 0.0249
Epoch   6 Batch  354/538 - Train Accuracy: 0.9824, Validation Accuracy: 0.9716, Loss: 0.0128
Epoch   6 Batch  356/538 - Train Accuracy: 0.9853, Validation Accuracy: 0.9741, Loss: 0.0120
Epoch   6 Batch  358/538 - Train Accuracy: 0.9809, Validation Accuracy: 0.9647, Loss: 0.0123
Epoch   6 Batch  360/538 - Train Accuracy: 0.9779, Validation Accuracy: 0.9616, Loss: 0.0133
Epoch   6 Batch  362/538 - Train Accuracy: 0.9883, Validation Accuracy: 0.9680, Loss: 0.0119
Epoch   6 Batch  364/538 - Train Accuracy: 0.9756, Validation Accuracy: 0.9689, Loss: 0.0182
Epoch   6 Batch  366/538 - Train Accuracy: 0.9832, Validation Accuracy

Epoch   6 Batch  524/538 - Train Accuracy: 0.9805, Validation Accuracy: 0.9666, Loss: 0.0142
Epoch   6 Batch  526/538 - Train Accuracy: 0.9728, Validation Accuracy: 0.9588, Loss: 0.0153
Epoch   6 Batch  528/538 - Train Accuracy: 0.9774, Validation Accuracy: 0.9627, Loss: 0.0145
Epoch   6 Batch  530/538 - Train Accuracy: 0.9787, Validation Accuracy: 0.9689, Loss: 0.0166
Epoch   6 Batch  532/538 - Train Accuracy: 0.9809, Validation Accuracy: 0.9666, Loss: 0.0138
Epoch   6 Batch  534/538 - Train Accuracy: 0.9808, Validation Accuracy: 0.9688, Loss: 0.0098
Epoch   6 Batch  536/538 - Train Accuracy: 0.9853, Validation Accuracy: 0.9707, Loss: 0.0154
Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [24]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.lower().split()]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [25]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [135, 23, 46, 167, 130, 189, 220]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [152, 18, 313, 242, 174, 353, 186, 350, 1]
  French Words: il a vu un vieux camion rouge . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.